### NLP Assignment-1

1.To run the code we can execute it cell by cell or Run all can also be used.  

2.It is Multilabel classification problem and macro F1 score is used for evaluation.

3.Many models executed are commented.We can try out by uncommenting them and comparing there scores.

Load data

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-09-25 14:10:14--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv.1’

train_split.csv.1   100%[===================>] 141.09K  --.-KB/s    in 0.02s   

2024-09-25 14:10:14 (6.12 MB/s) - ‘train_split.csv.1’ saved [144474/144474]

--2024-09-25 14:10:14--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (3

In [ ]:
# Install necessary libraries
!pip install pandas scikit-learn
!pip install nltk



Import Libraries and Create Data Frame


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import re

train = pd.read_csv('/content/train_split.csv')
test = pd.read_csv('/content/test_split.csv')

print(train.head())
print(test.head())


X_train = train['text']
y_train = train[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']]
X_test = test['text']
y_test = test[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']]





                                                text  Joy  Fear  Anger  \
0  The light would come all the way up to your ve...    0     1      0   
1                   Well, my birthday is in January.    0     0      0   
2  Tears in my eyes, too much on my mind, and dee...    0     1      0   
3  My eyes scanned quickly into every room, every...    0     1      1   
4  I felt the afterglow of the late afternoon sun...    1     0      0   

   Sadness  Surprise  
0        0         1  
1        0         0  
2        1         0  
3        0         0  
4        0         0  
                                                text  Joy  Fear  Anger  \
0  And they probably knew exactly what your dad saw.    0     1      0   
1  My neck still hurts while I'm working, and I w...    0     1      1   
2  and Christian fish bumper stickers, it caught ...    1     0      0   
3                               It felt so unreal...    0     1      0   
4  ' To be honest I don't know why I set out in

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier


# Regular expressions
def extract_sentiment_keywords(text):
    sentiment = {'Joy': 0, 'Fear': 0, 'Anger': 0, 'Sadness': 0, 'Surprise': 0}

    patterns = {
        'Joy': r'\b(?:[hH]app(?:y|iness|ily|ier|iest)?|[jJ]oy(?:ous|ful|fully)?|[eE]xcite(?:d|ment|s)?|[eE]lat(?:ed|ion)?|[pP]leas(?:ed|ure|ing)?|[dD]elight(?:ed|ful)?)\b',
        'Fear': r'\b(?:[fF]ear(?:ful|ing)?|[sS]care(?:d|y|s)?|[tT]errifi(?:ed|ing|es)?|[aA]fraid|worri(?:ed|es|some|ing)?)|[dD]ie|[dD]eath\b',
        'Anger': r'\b(?:[aA]ngr(?:y|ier|iest|ily)?|[mM]ad(?:ness)?|[fF]uri(?:ous|osity)?|[iI]rat(?:e|ion)?|[rR]esent(?:ful|ment)?)\b',
        'Sadness': r'\b(?:[sS]ad(?:ness|den|dest)?|[uU]nhapp(?:y|iness)?|[dD]epress(?:ed|ion)?|[mM]ourn(?:ful|ing)?|[gG]rief|[tT]ear(?:s|ful)?|[dD]ie|[dD]eath)\b',
        'Surprise': r'\b(?:[sS]urpris(?:ed|ing|es)?|[aA]maz(?:ed|ing|ement)?|[aA]stonish(?:ed|ing|ment)?|[sS]hock(?:ed|ing|s)?)\b',
    }

    for sentiment_type, pattern in patterns.items():
        if re.search(pattern, text, re.IGNORECASE):
            sentiment[sentiment_type] = 1
    return pd.Series(sentiment)


X_train_reg = X_train.apply(extract_sentiment_keywords)
X_test_reg = X_test.apply(extract_sentiment_keywords)


ngram_vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words='english')

X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)

# Convert n-gram features to DataFrame
ngram_features_train = pd.DataFrame(X_train_ngrams.toarray(), columns=ngram_vectorizer.get_feature_names_out())
ngram_features_test = pd.DataFrame(X_test_ngrams.toarray(), columns=ngram_vectorizer.get_feature_names_out())


X_train_combined = pd.concat([X_train_reg.reset_index(drop=True), ngram_features_train.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test_reg.reset_index(drop=True), ngram_features_test.reset_index(drop=True)], axis=1)

print(X_train_combined)
print(X_test_combined)

model = OneVsRestClassifier(LogisticRegression())


model.fit(X_train_combined, y_train)


y_pred_test_combined = model.predict(X_test_combined)

# F1 score
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"F1 Score (macro):", f1)


evaluate(y_test, y_pred_test_combined)


      Joy  Fear  Anger  Sadness  Surprise  00  00 started  00 started losing  \
0       0     0      0        0         0   0           0                  0   
1       0     0      0        0         0   0           0                  0   
2       0     0      0        1         0   0           0                  0   
3       0     0      0        0         0   0           0                  0   
4       0     0      0        0         0   0           0                  0   
...   ...   ...    ...      ...       ...  ..         ...                ...   
1595    0     0      0        0         0   0           0                  0   
1596    0     0      0        0         0   0           0                  0   
1597    0     0      0        0         0   0           0                  0   
1598    0     0      0        0         0   0           0                  0   
1599    0     0      0        0         0   0           0                  0   

      00 way  00 way gone  ...  zephyr 

In [ ]:
# # Using TFIDF Vectorizer

# # Text Preprocessing using TfidfVectorizer
# vectorizer = TfidfVectorizer(stop_words='english', max_features=15000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # One-vs-Rest for multilabel classification

# # Naive Bayes
# NB = OneVsRestClassifier(MultinomialNB())
# NB.fit(X_train_tfidf, y_train)
# NBprediction = NB.predict(X_test_tfidf)

# # Logistic Regression
# LR = OneVsRestClassifier(LogisticRegression(max_iter=1000))
# LR.fit(X_train_tfidf, y_train)
# LRpredictions = LR.predict(X_test_tfidf)

# # Random Forest
# RF = OneVsRestClassifier(RandomForestClassifier(random_state=42))
# RF.fit(X_train_tfidf, y_train)
# RFpredictions = RF.predict(X_test_tfidf)

# # SVM
# SVM = OneVsRestClassifier(SVC())
# SVM.fit(X_train_tfidf, y_train)
# SVMpredictions = SVM.predict(X_test_tfidf)


# # F1 Score
# NB_f1 = f1_score(y_test, NBprediction,average='macro')
# LR_f1 = f1_score(y_test, LRpredictions,average='macro')
# RF_f1 = f1_score(y_test, RFpredictions,average='macro')
# SVM_f1 = f1_score(y_test, SVMpredictions,average='macro')

# print("Naive Bayes F1 Score (macro):", NB_f1)
# print("Logistic Regression F1 Score (macro):", LR_f1)
# print("Random Forest F1 Score (macro):", RF_f1)
# print("SVM F1 Score (macro):", SVM_f1)

In [ ]:
# Using Count Vectorizer

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', max_features=15000)
X_train_count = vectorizer.fit_transform(X_train)
X_test_count = vectorizer.transform(X_test)

# # Naive Bayes
# NB = OneVsRestClassifier(MultinomialNB())
# NB.fit(X_train_count, y_train)
# NBprediction = NB.predict(X_test_count)

# # Logistic Regression
# LR = OneVsRestClassifier(LogisticRegression(max_iter=1000))
# LR.fit(X_train_count, y_train)
# LRpredictions = LR.predict(X_test_count)

# Random Forest
RF = OneVsRestClassifier(RandomForestClassifier(random_state=42))
RF.fit(X_train_count, y_train)
RFpredictions = RF.predict(X_test_count)

# # SVM
# SVM = OneVsRestClassifier(SVC())
# SVM.fit(X_train_count, y_train)
# SVMpredictions = SVM.predict(X_test_count)


# F1 Score
# NB_f1 = f1_score(y_test, NBprediction,average='macro')
# LR_f1 = f1_score(y_test, LRpredictions,average='macro')
RF_f1 = f1_score(y_test, RFpredictions,average='macro')
# SVM_f1 = f1_score(y_test, SVMpredictions,average='macro')

# print("Naive Bayes F1 Score (macro):", NB_f1)
# print("Logistic Regression F1 Score (macro):", LR_f1)
print("Random Forest F1 Score (macro):", RF_f1)
# print("SVM F1 Score (macro):", SVM_f1)

Random Forest F1 Score (macro): 0.396847796201511


Macro F1 score for Random Forest is highest and using CountVectorizer(Bag of Words) Vector Transformation.(0.396)